In [4]:
class Option():
    
    def __init__(self):
        self.dataroot= './data/RAFDB_train' #image dataroot
        self.coarse = './data/coarse_RAFDB_train'
        self.FERlabel = './data/RAFDB_train.txt' #FER label file
        self.maskroot= './data/mask'#mask dataroot
        self.batchSize= 1# Need to be set to 1
        self.fineSize=224 # image size
        self.input_nc=3  # input channel size for first stage
        self.input_nc_g=6 # input channel size for second stage
        self.output_nc=3# output channel size
        self.ngf=64 # inner channel
        self.ndf=64# inner channel
        self.which_model_netD='basic' # patch discriminator
        self.which_model_netF='feature'# feature patch discriminator
        self.which_model_netG='unet_assemble'#
        self.triple_weight=1
        self.name='hybrid_grad'
        self.n_layers_D='3'
        self.gpu_ids=[0]
        self.model='hybrid_net'
        self.checkpoints_dir='checkpoints/' #
        self.norm='instance'
        self.fixed_mask=1
        self.use_dropout=False
        self.init_type='normal'
        self.mask_type='random'
        self.lambda_A=100
        self.lambda_sc=100
        self.threshold=5/16.0
        self.stride=1
        self.shift_sz=1 # size of feature patch
        self.mask_thred=1
        self.bottleneck=512
        self.gp_lambda=10.0
        self.ncritic=5
        self.constrain='MSE'
        self.strength=1
        self.init_gain=0.02
        self.cosis=1
        self.gan_type='lsgan'
        self.gan_weight=0.2
        self.overlap=4
        self.skip=0
        self.display_freq=1000
        self.print_freq=50
        self.save_latest_freq=5000
        self.save_epoch_freq=2
        self.continue_train=False
        self.epoch_count=1
        self.phase='train'
        self.which_epoch=''
        self.niter=20 
        self.niter_decay=100
        self.beta1=0.5
        self.lr=0.0002
        self.lr_policy='lambda'
        self.lr_decay_iters=50
        self.isTrain=True

In [5]:
import time
from util.data_load import Data_load
from models.models import create_model
from models.FER import FER
import torch
import os
import torchvision
from torch.utils import data
import torchvision.transforms as transforms
opt = Option()
transform_mask = transforms.Compose(
    [transforms.Resize((256,256)),
     transforms.ToTensor(),
    ])
transform = transforms.Compose(
    [transforms.Resize((256,256)),
     transforms.ToTensor(),
     transforms.Normalize(mean=[0.5] * 3, std=[0.5] * 3)])

dataset_train = Data_load('train',opt.dataroot,opt.coarse, opt.maskroot, transform, transform_mask,opt.FERlabel)
iterator_train = (data.DataLoader(dataset_train, batch_size=1,shuffle=True))
print(len(dataset_train))
model_fer = FER()
checkpoint = torch.load('./models/fer_model/OURS_Combine_epoch13_acc0.7255.pth')
model_fer.load_state_dict(checkpoint['model_state_dict'],strict=True)
model_fer.cuda()
model = create_model(opt,model_fer)
total_steps = 0

16
hybrid_net
initialize network with normal
initialize network with normal
initialize network with normal
---------- Networks initialized -------------
UnetGeneratorHybrid(
  (model): UnetSkipConnectionBlock_3_outer(
    (model): Sequential(
      (0): Conv2d(12, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): UnetSkipConnectionBlock_3_outer(
        (model): Sequential(
          (0): LeakyReLU(negative_slope=0.2, inplace=True)
          (1): Conv2d(128, 128, kernel_size=(4, 4), stride=(2, 2), padding=(3, 3), dilation=(2, 2))
          (2): InstanceNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
          (3): LeakyReLU(negative_slope=0.2, inplace=True)
          (4): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (5): InstanceNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
          (6): UnetSkipConnectionBlock_3_outer(
            (model): Sequential(
              (0): Leaky

In [6]:
iter_start_time = time.time()
save_dir = "./train_result_lambda100/"
for epoch in range(opt.epoch_count, opt.niter + opt.niter_decay + 1):   
    epoch_start_time = time.time()
    epoch_iter = 0


    for image,coarse_image,mask,name,label in (iterator_train):
        image=image.cuda()
        mask=mask.cuda()
        mask=mask[0][0]
        mask=torch.unsqueeze(mask,0)
        mask=torch.unsqueeze(mask,1)
        mask=mask.byte()

        label = label.cuda()

        total_steps += opt.batchSize
        epoch_iter +=opt.batchSize
        model.set_input(image,coarse_image,mask,label) 
        model.set_gt_latent()
        model.optimize_parameters()
        if total_steps %opt.display_freq== 0:
            real_A,real_B,coarse,fake_B=model.get_current_visuals()
            
            pic = (torch.cat([real_A, real_B,coarse,fake_B], dim=0) + 1) / 2.0
            torchvision.utils.save_image(pic, '%s/Epoch_(%d)_(%dof%d)_%s.jpg' % (
            save_dir, epoch, total_steps + 1, len(dataset_train),name), nrow=2)
        if total_steps %1== 0:
            errors = model.get_current_errors()
            t = (time.time() - iter_start_time) / opt.batchSize
            print(errors)
    
    if epoch % opt.save_epoch_freq == 0:
        print('saving the model at the end of epoch %d, iters %d' %(epoch, total_steps))
        model.save(epoch)

    print('End of epoch %d / %d \t Time Taken: %d sec' %
            (epoch, opt.niter + opt.niter_decay, time.time() - epoch_start_time))

    model.update_learning_rate()


D:\VILAB\assembly\models\hybrid.py:122: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead. (Triggered internally at  C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\cuda\Indexing.cu:1240.)
  self.input_A.narrow(1,0,1).masked_fill_(self.mask_global, 2*123.0/255.0 - 1.0)
D:\VILAB\assembly\models\hybrid.py:123: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead. (Triggered internally at  C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\cuda\Indexing.cu:1240.)
  self.input_A.narrow(1,1,1).masked_fill_(self.mask_global, 2*104.0/255.0 - 1.0)
D:\VILAB\assembly\models\hybrid.py:124: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead. (Triggered internally at  C:\cb\pytorch_1000000000000\work\aten\src\A

OrderedDict([('G_GAN', 14.081199645996094), ('G_L1', 66.3862533569336), ('G_SC', 1.7382941246032715), ('G_C1', tensor(0.7334, device='cuda:0', grad_fn=<AddBackward0>)), ('G_C2', tensor(0.9229, device='cuda:0', grad_fn=<AddBackward0>)), ('D', 2.4638850688934326), ('F', 2.369035482406616)])
OrderedDict([('G_GAN', 12.203937530517578), ('G_L1', 53.38641357421875), ('G_SC', 1.704145908355713), ('G_C1', tensor(0.4784, device='cuda:0', grad_fn=<AddBackward0>)), ('G_C2', tensor(0.7031, device='cuda:0', grad_fn=<AddBackward0>)), ('D', 5.820974349975586), ('F', 1.8655552864074707)])
OrderedDict([('G_GAN', 6.140406131744385), ('G_L1', 39.617149353027344), ('G_SC', 1.8430132865905762), ('G_C1', tensor(0.9811, device='cuda:0', grad_fn=<AddBackward0>)), ('G_C2', tensor(1.1572, device='cuda:0', grad_fn=<AddBackward0>)), ('D', 3.4443726539611816), ('F', 1.7842979431152344)])
OrderedDict([('G_GAN', 11.060735702514648), ('G_L1', 44.084228515625), ('G_SC', 1.6002953052520752), ('G_C1', tensor(0.3480, dev

KeyboardInterrupt: 